# Engine Earth

## Google Earth Engine (GEE)

In [1]:
import ee
import geemap
from pathlib import Path
import geopandas as gpd
import pandas as pd

nb_dir = Path.cwd()
repo_root = nb_dir.parent  # adjust if needed


/opt/anaconda3/envs/gee310/lib/python3.10/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# Confirm authentication
ee.Authenticate(auth_mode='notebook')
print("Authentification:" ,ee.Authenticate())

# If Authentication issue, force in CLI
    # earthengine authenticate --quiet --force

# Initialize the Earth Engine module.
google_project = "wildfire-canada-475322"
ee.Initialize(project=str(google_project))
print("Initialized:", ee.data._credentials is not None, f"with {google_project}")
print(f"Initialized With Google Project {google_project}")


# Print current EE version
print("Earth Engine Version: ",ee.__version__)

# Test data access
print("Data Access:", ee.Number(1).getInfo())


Authentification: True
Initialized: True with wildfire-canada-475322
Initialized With Google Project wildfire-canada-475322
Earth Engine Version:  0.1.419
Data Access: 1


In [18]:
BC_fires_path = repo_root / "data/processed/shapefiles/BC_fires_2024.geojson"

gdf_bc = gpd.read_file(BC_fires_path)
gdf_bc
gdf_bc_4326 = gdf_bc.to_crs(epsg=4326)
out_dir = "BC_fires_2024_shp"
gdf_bc_4326.to_file(out_dir) 

In [16]:
row_one = gdf_bc.iloc[[0]]
row_one.to_csv('test.csv')

In [14]:


# 1. Pick one fire (original geometry)
fire_rows = gdf_bc.iloc[[0]]                 # keep original polygon
fire_fc   = geemap.geopandas_to_ee(fire_rows)  # ee.FeatureCollection
fire_geom = fire_fc.geometry()

# 2. Build a bounding box around that fire
box_rows = fire_rows.copy()
box_rows["geometry"] = box_rows.geometry.envelope   # shapely envelope
box_fc   = geemap.geopandas_to_ee(box_rows)
box_geom = box_fc.geometry()                        # this is the bbox geom

# 3. Sentinel-2 image collection, bounded by the box (smaller request)
Sen2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

visParams = {
    "bands": ['B4', 'B3', 'B2'],
    "min": 0,
    "max": 3000,
}

BcFire = (
    Sen2
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
    .filterBounds(box_geom)                                     # use bbox here
    .filterDate('2019-01-01', '2020-12-31')
)

BcFireComposite = BcFire.median().clip(box_geom)                # clip to bbox

# 4. Map
Map = geemap.Map()
Map.centerObject(box_geom, 11)

# 5. Style the **original polygon** as an OUTLINE only
outline_style = {
    "color": "FF0000",     # red outline
    "fillColor": "00000000",  # fully transparent
    "width": 1,
}

outline = fire_fc.style(**outline_style)        # style the FeatureCollection

# 6. Add layers
Map.addLayer(BcFireComposite, visParams, "S2 bbox composite")
Map.addLayer(outline, {}, "Fire outline")

Map


Map(center=[58.199352120055394, -121.7146462583415], controls=(WidgetControl(options=['position', 'transparent…

In [6]:
rows = gdf_bc.iloc[[0]]

row_box = rows.copy()
# row_box['geometry'] = row_box['geometry'].envelope  # shapely bounding box

fc_bc = geemap.geopandas_to_ee(row_box)              # ee.FeatureCollection
geom_bc = fc_bc.geometry()                       # union geometry if needed




Sen2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

visParams = {
    "bands": ['B4', 'B3', 'B2'],
    "min": 0,
    "max": 3000,
}

BcFire = (
    Sen2
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
    .filter(ee.Filter.bounds(geom_bc))
    .filter(ee.Filter.date('2019-01-01', '2020-12-31'))
)

BcFireComposite = BcFire.median()



Map = geemap.Map()
Map.centerObject(geom_bc, 12)


# Add the Sentinel-2 composite clipped to this single fire

Map.addLayer(BcFireComposite.clip(geom_bc), visParams, "BC fire 159")
Map

Map(center=[58.21445951281796, -121.6981956475732], controls=(WidgetControl(options=['position', 'transparent_…